In [1]:
from pathlib import Path
from time import sleep 

import numpy as np

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator import VisualRobot
from manipylator import utils

# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
# path = Path('/workspace/robots/robot.urdf')
path = Path('/workspace/robots/measured/robot.urdf')
manny = VisualRobot(path)
print(manny.model)

[I 05/20/25 01:30:51.710 795] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout
[Genesis] [01:30:56] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [01:30:56] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [01:30:56] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [01:30:56] [INFO] Running on [NVIDIA GeForce MX250] with backend gs.cuda. Device memory: 1.95 GB.
[Genesis] [01:30:57] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [01:30:58] [INFO] Scene <39fbbbb> created.
[Genesis] [01:30:58] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <3707099>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [01:30:58] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <0044635>, morph: <gs.morphs.URDF(file='/workspace/robots/measured/robot.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [01:31:01] [INFO] Building

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space. Below we find the location of the end effector in when the all joint parameters are set to 0 degrees:

In [3]:
robot = manny.visualizer.robot
# robot

In [4]:
ee = robot.get_link('end_effector')
# ee

In [5]:
ee_initial_pos = ee.get_pos()
ee_initial_pos

tensor([-1.2851e-01,  1.5981e-06,  8.1101e-01], device='cuda:0')

In [6]:
ee_initial_quat = ee.get_quat()
ee_initial_quat

tensor([0.5000, 0.5000, 0.5000, 0.5000], device='cuda:0')

And we can verify that for the inverse is also true:

In [7]:
initial_qpos = robot.inverse_kinematics(
    link = ee,
    pos  = ee_initial_pos,
    quat = ee_initial_quat,
)
initial_qpos

[W 05/20/25 01:32:33.459 795] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')

I.e. that when setting all the joints to 0 degrees the end effector end up at `ee_initial_pos` and that if the end effector is at `ee_initial_pos` all the joints are at 0 degress.

Similarly, for an arbitrary (reachable) location in 3D space:

In [8]:
qpos = robot.inverse_kinematics(
    link = ee,
    pos  = [-4.8500e-02, -4.7547e-02,  4.0200e-01],
    quat = ee_initial_quat,
)
qpos

tensor([-3.1369,  0.4008,  2.0764,  3.1113,  3.1410,  1.4357], device='cuda:0')

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [9]:
scale = 0.1
offset = 0.3
height = 0.1

shape = utils.parametric_heart_1(np.linspace(0, 2*np.pi, 100))
positions = scale * shape + offset
positions[:,2] = height

positions[:3]

array([[0.3       , 0.3       , 0.1       ],
       [0.30003608, 0.30060238, 0.1       ],
       [0.30028691, 0.30238772, 0.1       ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [10]:
# for x, y, z in positions:
#     manny.visualizer.scene.draw_debug_sphere([x,y,z])
manny.visualizer.scene.draw_debug_spheres(positions)



In [11]:
# manny.visualizer.scene.clear_debug_objects()

Similarly, we can calculate the joint parameters of each position:

In [27]:
robot.set_dofs_position([0.2, 0.2, 2.35, 1, 1.57, 0])

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0357,  0.2696,  0.2888], device='cuda:0')

In [19]:
quat_pointing_down = [0, -1, 0, 0]

initial_pose = robot.inverse_kinematics(
        link = ee,
        pos  = positions[0],
        quat = quat_pointing_down,
        )

robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

In [20]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()

In [21]:
camera = manny.visualizer.camera

camera.start_recording()
# [camera.render() for i in range(10)]

# Reset to initial curve pose
robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

camera.render()

(array([[[35, 53, 71],
         [35, 53, 71],
         [35, 53, 71],
         ...,
         [38, 55, 74],
         [39, 57, 76],
         [41, 58, 77]],
 
        [[34, 52, 70],
         [34, 51, 70],
         [33, 51, 69],
         ...,
         [42, 59, 78],
         [42, 60, 78],
         [42, 60, 79]],
 
        [[26, 44, 62],
         [26, 44, 62],
         [26, 44, 62],
         ...,
         [38, 56, 75],
         [37, 54, 73],
         [35, 53, 71]],
 
        ...,
 
        [[21, 37, 53],
         [21, 37, 53],
         [21, 37, 53],
         ...,
         [22, 39, 55],
         [22, 39, 55],
         [22, 39, 55]],
 
        [[21, 37, 53],
         [21, 37, 53],
         [21, 37, 53],
         ...,
         [22, 39, 55],
         [22, 39, 55],
         [22, 39, 55]],
 
        [[21, 37, 53],
         [21, 37, 53],
         [21, 37, 53],
         ...,
         [22, 39, 55],
         [22, 39, 55],
         [22, 39, 55]]], dtype=uint8),
 None,
 None,
 None)

In [24]:

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = quat_pointing_down,
        ) for pos in positions]
qs[:5]

[tensor([ 0.9956, -2.4955, -2.0744, -1.1497, -1.5707,  2.1461], device='cuda:0'),
 tensor([ 0.9944, -2.4949, -2.0727, -1.1485, -1.5707,  2.1472], device='cuda:0'),
 tensor([ 0.9912, -2.4936, -2.0677, -1.1449, -1.5707,  2.1507], device='cuda:0'),
 tensor([ 0.9862, -2.4893, -2.0568, -1.1383, -1.5708,  2.1554], device='cuda:0'),
 tensor([ 0.9802, -2.4843, -2.0424, -1.1289, -1.5708,  2.1614], device='cuda:0')]

And to visualize the poses Manny would take, we can animate the path:

In [25]:
for q in qs:
    robot.set_dofs_position(q)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    # camera.render()
    sleep(0.1)

camera.stop_recording(save_to_filename='video.mp4', fps=20)

[Genesis] [01:34:45] [INFO] Saving video to video.mp4.
[Genesis] [01:34:45] [INFO] Video saved.


In [23]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()
